<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/spark_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz
!pip install -q findspark

hadoop to store data
advise executing this directly in google colab to avoid installing spark in your machine

env variables to have together spark and python

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

where the magic begins
entity of spark that can be used with python


In [5]:
import findspark
findspark.init("spark-3.2.3-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
# spark content, to invoke 'methods' you want to use

In [7]:
sc = spark.sparkContext

In [ ]:
# from now on examples of basic operations
# a string, parallelizing the split of this string
# ''' string''' -> 3 strings

In [8]:
s = '''Every person had a star, every star had a friend, and for every person
carrying a star there was someone else who reflected it, and everyone
carried this reflection like a secret confidante in the heart'''
simple_rdd = sc.parallelize(s.split('\n'))

In [9]:
simple_rdd.collect()

['Every person had a star, every star had a friend, and for every person',
 'carrying a star there was someone else who reflected it, and everyone',
 'carried this reflection like a secret confidante in the heart']

In [ ]:
# applied on each instance of the RDD

In [10]:
(simple_rdd.map(lambda line: line.split(' '))
           .collect())

[['Every',
  'person',
  'had',
  'a',
  'star,',
  'every',
  'star',
  'had',
  'a',
  'friend,',
  'and',
  'for',
  'every',
  'person'],
 ['carrying',
  'a',
  'star',
  'there',
  'was',
  'someone',
  'else',
  'who',
  'reflected',
  'it,',
  'and',
  'everyone'],
 ['carried',
  'this',
  'reflection',
  'like',
  'a',
  'secret',
  'confidante',
  'in',
  'the',
  'heart']]

In [11]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .take(10))

['Every',
 'person',
 'had',
 'a',
 'star,',
 'every',
 'star',
 'had',
 'a',
 'friend,']

In [12]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .take(5))

['every', 'person', 'had', 'a', 'star']

In [13]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .map(lambda word: (word, 1))
           .take(5))

[('every', 1), ('person', 1), ('had', 1), ('a', 1), ('star', 1)]

In [14]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(',', '').lower())
           .map(lambda word: (word, 1))
           .reduceByKey(lambda a,b: a+b)
           .collect())

[('person', 2),
 ('there', 1),
 ('was', 1),
 ('carried', 1),
 ('this', 1),
 ('like', 1),
 ('secret', 1),
 ('confidante', 1),
 ('in', 1),
 ('heart', 1),
 ('every', 3),
 ('had', 2),
 ('a', 4),
 ('star', 3),
 ('friend', 1),
 ('and', 2),
 ('for', 1),
 ('carrying', 1),
 ('someone', 1),
 ('else', 1),
 ('who', 1),
 ('reflected', 1),
 ('it', 1),
 ('everyone', 1),
 ('reflection', 1),
 ('the', 1)]

In [15]:
def count_freq(rdd):
  return (rdd.flatMap(lambda line: line.split(' '))
            .map(lambda word: word.replace(',', '').lower())
            .map(lambda word: (word, 1))
            .reduceByKey(lambda a,b: a+b)
            .collect())

In [16]:
!wget http://www.scifiscripts.com/scripts/swd1_5-74.txt

--2023-01-30 13:08:37--  http://www.scifiscripts.com/scripts/swd1_5-74.txt
Resolving www.scifiscripts.com (www.scifiscripts.com)... 207.32.177.145
Connecting to www.scifiscripts.com (www.scifiscripts.com)|207.32.177.145|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-01-30 13:08:38 ERROR 403: Forbidden.



In [ ]:
sw = sc.textFile('swd1_5-74.txt')

In [ ]:
sw.take(10)

['The Star Wars',
 'by',
 'George Lucas',
 '',
 '',
 '',
 'Rough Draft [First of four major screenplay drafts]',
 'Lucasfilm Ltd.',
 '5/74',
 '']

In [ ]:
count_freq(sw)[: 10]

[('george', 1),
 ('', 3543),
 ('draft', 1),
 ('of', 744),
 ('four', 25),
 ('major', 2),
 ('drafts]', 1),
 ('ltd.', 1),
 ('5/74', 1),
 ('1.', 1)]

In [ ]:
import random

NUM_SAMPLES = 10**7

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = (sc.parallelize(range(0, NUM_SAMPLES))
           .filter(inside).count())

print('Pi is roughly {}'.format(4.0 * count / NUM_SAMPLES))

Pi is roughly 3.1404776
